In [26]:
import numpy as np
import pandas as pd


In [27]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
users = pd.read_csv('users.csv')
tags = pd.read_csv('tags.csv')

In [28]:
movies.head()

,mid,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [29]:
ratings.head()

,mid,rating
0,1,4.0
1,3,4.0
2,6,4.0
3,47,5.0
4,50,5.0


In [30]:
users.head()

,uid,mid,timestamp
0,1,1,964982703
1,1,3,964981247
2,1,6,964982224
3,1,47,964983815
4,1,50,964982931


In [31]:
print(movies.shape)

(9742, 3)


In [32]:
print(ratings.shape)

(100836, 2)


In [33]:
movies.isnull().sum()

mid       0
title     0
genres    0
dtype: int64

###WE HAVE TAKEN SUCH DATASET OF MOVIES WHERE EVERY MOVIE HOLDS A TITLE FOR SURE ALONG WITH A GENRE

In [34]:
ratings.isnull().sum()

mid       0
rating    0
dtype: int64

In [35]:
users.isnull().sum()

uid          0
mid          0
timestamp    0
dtype: int64

##POPULARITY BASED VIA COLLABORATIVE FILTERING RECOMMENDATIONS FILTERING

##THE MOVIES WITH HIGHEST RATING AND RATED BY MOST PEOPLE WILL OCCUR AT TOP

In [36]:
ratings_with_title = ratings.merge(movies,on='mid')

In [37]:
ratings_with_title

,mid,rating,title,genres
0,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...
100831,160341,2.5,Bloodmoon (1997),Action|Thriller
100832,160527,4.5,Sympathy for the Underdog (1971),Action|Crime|Drama
100833,160836,3.0,Hazard (2005),Action|Drama|Thriller
100834,163937,3.5,Blair Witch (2016),Horror|Thriller


##THIS SHOWS MULTIPLE MOVIES MOVIES GOT RATINGS FROM MORE THAN 1 USER

In [38]:
ratings_with_title.groupby('title').count()

,mid,rating,genres
title,,,
'71 (2014),1,1,1
'Hellboy': The Seeds of Creation (2004),1,1,1
'Round Midnight (1986),2,2,2
'Salem's Lot (2004),1,1,1
'Til There Was You (1997),2,2,2
...,...,...,...
eXistenZ (1999),22,22,22
xXx (2002),24,24,24
xXx: State of the Union (2005),5,5,5


##THE ABOVE SHOWS A SINGLE MOVE CAN HAVE MULTIPLE NUMBER OF RATINGS BY SEPERATE USERS EACH MOVIE HAS GOT.

In [39]:
ratings_with_title.groupby('title').count()['rating'].reset_index()

,title,rating
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2
...,...,...
9714,eXistenZ (1999),22
9715,xXx (2002),24
9716,xXx: State of the Union (2005),5
9717,¡Three Amigos! (1986),26


In [40]:
num_ratings_tit = ratings_with_title.groupby('title').count()['rating'].reset_index()
num_ratings_tit.rename(columns={'rating' : 'num_of_rating'},inplace=True)
num_ratings_tit

,title,num_of_rating
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2
...,...,...
9714,eXistenZ (1999),22
9715,xXx (2002),24
9716,xXx: State of the Union (2005),5
9717,¡Three Amigos! (1986),26


##The above shows a seperate sheet made for only the title and number  of ratings with a single index provided to each number

In [41]:
avg_ratings_tit = ratings_with_title.groupby('title')['rating'].mean().reset_index()
avg_ratings_tit.rename(columns={'rating' : 'avg_of_ratings'},inplace=True)
avg_ratings_tit

,title,avg_of_ratings
0,'71 (2014),4.000000
1,'Hellboy': The Seeds of Creation (2004),4.000000
2,'Round Midnight (1986),3.500000
3,'Salem's Lot (2004),5.000000
4,'Til There Was You (1997),4.000000
...,...,...
9714,eXistenZ (1999),3.863636
9715,xXx (2002),2.770833
9716,xXx: State of the Union (2005),2.000000
9717,¡Three Amigos! (1986),3.134615


In [42]:
most_popular=num_ratings_tit.merge(avg_ratings_tit,on='title')
most_popular

,title,num_of_rating,avg_of_ratings
0,'71 (2014),1,4.000000
1,'Hellboy': The Seeds of Creation (2004),1,4.000000
2,'Round Midnight (1986),2,3.500000
3,'Salem's Lot (2004),1,5.000000
4,'Til There Was You (1997),2,4.000000
...,...,...,...
9714,eXistenZ (1999),22,3.863636
9715,xXx (2002),24,2.770833
9716,xXx: State of the Union (2005),5,2.000000
9717,¡Three Amigos! (1986),26,3.134615


In [43]:
most_popular[most_popular['num_of_rating']>=50]

,title,num_of_rating,avg_of_ratings
18,10 Things I Hate About You (1999),54,3.527778
34,12 Angry Men (1957),57,4.149123
74,2001: A Space Odyssey (1968),109,3.894495
89,28 Days Later (2002),58,3.974138
104,300 (2007),80,3.681250
...,...,...,...
9621,X2: X-Men United (2003),76,3.723684
9651,You've Got Mail (1998),50,3.120000
9656,Young Frankenstein (1974),69,3.992754
9699,Zombieland (2009),53,3.877358


##THE ABOVE SHOWS THE MOVIES WHICH WERE RATED MORE THAN 50 TIMES.

##NOW WE SORT THE ABOVE MOVIES AS PER THE HIGHEST RARINGS RECEIVED

In [44]:
most_popular[most_popular['num_of_rating']>=50].sort_values('avg_of_ratings', ascending=False)

,title,num_of_rating,avg_of_ratings
7593,"Shawshank Redemption, The (1994)",317,4.429022
3499,"Godfather, The (1972)",192,4.289062
3011,Fight Club (1999),218,4.272936
1961,Cool Hand Luke (1967),57,4.271930
2531,Dr. Strangelove or: How I Learned to Stop Worr...,97,4.268041
...,...,...,...
4606,Johnny Mnemonic (1995),53,2.679245
4634,Judge Dredd (1995),62,2.669355
1799,City Slickers II: The Legend of Curly's Gold (...,55,2.645455
1932,Coneheads (1993),63,2.420635


##SINCE WE CAN not HANDLE ADDING database , links and posters of all of the above so we consider only top 250 films

In [45]:
most_popular_final = most_popular[most_popular['num_of_rating']>=50].sort_values('avg_of_ratings', ascending=False).head(250)

In [46]:
most_popular_final

,title,num_of_rating,avg_of_ratings
7593,"Shawshank Redemption, The (1994)",317,4.429022
3499,"Godfather, The (1972)",192,4.289062
3011,Fight Club (1999),218,4.272936
1961,Cool Hand Luke (1967),57,4.271930
2531,Dr. Strangelove or: How I Learned to Stop Worr...,97,4.268041
...,...,...,...
104,300 (2007),80,3.681250
2663,Ed Wood (1994),70,3.678571
8644,There's Something About Mary (1998),105,3.676190
5029,Lethal Weapon (1987),75,3.673333


In [47]:
most_popular_final.merge(movies,on='title')

,title,num_of_rating,avg_of_ratings,mid,genres
0,"Shawshank Redemption, The (1994)",317,4.429022,318,Crime|Drama
1,"Godfather, The (1972)",192,4.289062,858,Crime|Drama
2,Fight Club (1999),218,4.272936,2959,Action|Crime|Drama|Thriller
3,Cool Hand Luke (1967),57,4.271930,1276,Drama
4,Dr. Strangelove or: How I Learned to Stop Worr...,97,4.268041,750,Comedy|War
...,...,...,...,...,...
245,300 (2007),80,3.681250,51662,Action|Fantasy|War|IMAX
246,Ed Wood (1994),70,3.678571,235,Comedy|Drama
247,There's Something About Mary (1998),105,3.676190,1923,Comedy|Romance
248,Lethal Weapon (1987),75,3.673333,2000,Action|Comedy|Crime|Drama


In [48]:
final_with_genre = most_popular_final.merge(movies,on='title').drop_duplicates('title')

In [49]:
final_with_genre

,title,num_of_rating,avg_of_ratings,mid,genres
0,"Shawshank Redemption, The (1994)",317,4.429022,318,Crime|Drama
1,"Godfather, The (1972)",192,4.289062,858,Crime|Drama
2,Fight Club (1999),218,4.272936,2959,Action|Crime|Drama|Thriller
3,Cool Hand Luke (1967),57,4.271930,1276,Drama
4,Dr. Strangelove or: How I Learned to Stop Worr...,97,4.268041,750,Comedy|War
...,...,...,...,...,...
245,300 (2007),80,3.681250,51662,Action|Fantasy|War|IMAX
246,Ed Wood (1994),70,3.678571,235,Comedy|Drama
247,There's Something About Mary (1998),105,3.676190,1923,Comedy|Romance
248,Lethal Weapon (1987),75,3.673333,2000,Action|Comedy|Crime|Drama


In [50]:
final_with_genre.merge(tags,on='mid').drop_duplicates('title')

,title,num_of_rating,avg_of_ratings,mid,genres,tag,timestamp
0,"Shawshank Redemption, The (1994)",317,4.429022,318,Crime|Drama,scifi,1525554839
1,"Godfather, The (1972)",192,4.289062,858,Crime|Drama,mindfuck,1348626910
2,Fight Club (1999),218,4.272936,2959,Action|Crime|Drama|Thriller,family,1138039618
3,Cool Hand Luke (1967),57,4.271930,1276,Drama,Myth,1436944141
4,Dr. Strangelove or: How I Learned to Stop Worr...,97,4.268041,750,Comedy|War,Coen Brothers,1300485477
...,...,...,...,...,...,...,...
245,300 (2007),80,3.681250,51662,Action|Fantasy|War|IMAX,Zooey Deschanel,1353384717
246,Ed Wood (1994),70,3.678571,235,Comedy|Drama,Charlize Theron,1525554121
247,There's Something About Mary (1998),105,3.676190,1923,Comedy|Romance,Vietnam,1138137933
248,Lethal Weapon (1987),75,3.673333,2000,Action|Comedy|Crime|Drama,Henry James,1138031883
